<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Get-files" data-toc-modified-id="Get-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get files</a></span></li><li><span><a href="#Extracting-features" data-toc-modified-id="Extracting-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting features</a></span></li><li><span><a href="#Export-DataFrame-to-.CSV" data-toc-modified-id="Export-DataFrame-to-.CSV-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export DataFrame to .CSV</a></span></li></ul></div>

# Libraries

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
# For audio playing
import IPython.display as ipd
# For visualizations
import librosa.display
from math import ceil

# Get files 

**To maximize the results obtained from the data and ensure that all samples have the same duration we will use the following function.**

1. Navigate through directories to get files.</li>
2. For each file we get several fragments that overlap the previous and following fragment.</li>


In [9]:
directory = "data"
def gen_audio(window,overlap=0,path=directory):
    for genre in os.listdir(path):
        if os.path.isdir(f"{path}/{genre}"):
            folder=f"{path}/{genre}"
            for song in os.listdir(folder):
                song_path = f"{folder}/{song}"
                if song == ".DS_Store":
                    continue
                y, sr = librosa.load(song_path)
                duration = True
                i=0
                while duration:
                    #print(song_path)
                    song_points = len(y)
                    window_points = window*sr
                    start = int(i*window_points - overlap*window_points)
                    if start<0:
                        start=0
                    end = start + window_points
                    #print(start,end)
                    sample=y[start:end]
                    if end >= song_points:
                        #print("-")
                        sample = y[-window_points:]
                        duration = False
                    i += 1
                    yield genre,sample,sr

In [10]:
audio_generator = gen_audio(15,0.25)

In [11]:
df = pd.DataFrame()

# Extracting features

**The features that are going to be used are:**
- Zero crossings
- Spectral centroid
- Spectral band_width
- Spectral rolloff
- Chroma stft
- Mel Frequency Cepstral Coeﬃcients (mfcc)
- Tonal centroid
- Onset_strength
- PLP
- Beats per min

In [12]:
%%time
for label, sample, sr in audio_generator:
    y = sample
    zero_crossing = librosa.zero_crossings(y=y, pad=False)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_band_width = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    tonal_centroid = librosa.feature.tonnetz(y=y, sr=sr)
    onset_strength = librosa.onset.onset_strength(y=y, sr=sr)
    plp = librosa.beat.plp(onset_envelope=onset_strength, sr=sr)
    beats = librosa.beat.beat_track(y=y, sr=sr)
    df = df.append({
        "label":label,
        "sample":sample,
        "sr":sr,
        "zero_crossings": sum(zero_crossing),
        "spectral_centroid": np.mean(spectral_centroid),
        "spectral_band_width": np.mean(spectral_band_width),
        "spectral_rolloff" : np.mean(spectral_rolloff),
        "chroma_stft": [np.mean(c) for c in chroma_stft],
        "mfcc": [np.mean(e) for e in mfcc],
        "tonal_centroid": [np.mean(t) for t in chroma_stft],
        "onset_strength" : np.mean(onset_strength),
        "plp" : np.mean(plp),
        "beats" : np.mean(np.mean(beats))
    }, ignore_index=True)

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 1h 14min 10s, sys: 20min 38s, total: 1h 34min 49s
Wall time: 44min 25s


In [13]:
df.head(1)

,beats,chroma_stft,label,mfcc,onset_strength,plp,sample,spectral_band_width,spectral_centroid,spectral_rolloff,sr,tonal_centroid,zero_crossings
0,189.402167,"[0.45334435, 0.39936936, 0.44807765, 0.5161125...",Supernatural,"[-242.92453, 136.92271, -24.217505, 26.947405,...",1.447724,0.273632,"[3.5482785e-08, 1.0161084e-08, -9.557592e-09, ...",1507.378098,1522.945223,2815.416309,22050.0,"[0.45334435, 0.39936936, 0.44807765, 0.5161125...",29991.0


In [14]:
df[['chroma_stft1','chroma_stft2','chroma_stft3','chroma_stft4','chroma_stft5','chroma_stft6','chroma_stft7','chroma_stft8','chroma_stft9','chroma_stft10','chroma_stft11','chroma_stft12']] = pd.DataFrame(df.chroma_stft.tolist(), index= df.index)
df = df.drop(['chroma_stft'], axis=1)

In [15]:
df[['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20']] = pd.DataFrame(df.mfcc.tolist(), index= df.index)
df = df.drop(['mfcc'], axis=1)

In [20]:
df['tonal_centroid'].iloc[0]

[0.45334435,
 0.39936936,
 0.44807765,
 0.5161125,
 0.6047815,
 0.6462182,
 0.5601424,
 0.42375207,
 0.4746412,
 0.29928055,
 0.2602959,
 0.29277796]

In [21]:
df[['tc1','tc2','tc3','tc4','tc5','tc6','tc7','tc8','tc9','tc10','tc11','tc12']] = pd.DataFrame(df.tonal_centroid.tolist(), index= df.index)
df = df.drop(['tonal_centroid'], axis=1)

In [22]:
df.head(1)

,beats,label,onset_strength,plp,sample,spectral_band_width,spectral_centroid,spectral_rolloff,sr,zero_crossings,...,tc3,tc4,tc5,tc6,tc7,tc8,tc9,tc10,tc11,tc12
0,189.402167,Supernatural,1.447724,0.273632,"[3.5482785e-08, 1.0161084e-08, -9.557592e-09, ...",1507.378098,1522.945223,2815.416309,22050.0,29991.0,...,0.448078,0.516113,0.604782,0.646218,0.560142,0.423752,0.474641,0.299281,0.260296,0.292778


In [23]:
df.shape

(1589, 54)

# Export DataFrame to .CSV

In [24]:
df.to_csv(path_or_buf= "output/even_more_features_15s_025ov.csv",index=False)